In [1]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from plotly.subplots import make_subplots



In [4]:
df = pd.read_csv(r"C:\Users\lenovo\Downloads\archive (8).zip") 
df


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,28-09-2012,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,05-10-2012,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,12-10-2012,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,19-10-2012,718125.53,0,56.47,3.969,192.330854,8.667


In [ ]:
#DATA CLEANING

In [6]:
df.shape

(6435, 8)

In [8]:
df = df.sample(n=6000).drop('Store',axis=1).reset_index(drop = True)

In [9]:
df.shape

(6000, 7)

In [18]:
df['CPI']= df['CPI'].round(2)
df['Unemployment']= df['Unemployment'].round(2)
df['Fuel_Price']= df['Fuel_Price'].round(2)
df

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,18-06-2010,815130.50,0,72.62,2.78,182.44,8.90
1,20-01-2012,897027.44,0,29.16,3.54,140.81,7.50
2,03-09-2010,1095932.51,0,70.20,2.77,190.36,9.02
3,17-12-2010,238875.26,0,61.12,3.12,126.88,9.26
4,31-12-2010,774262.28,1,28.49,3.18,136.67,8.57
...,...,...,...,...,...,...,...
5995,21-10-2011,1203172.05,0,73.05,3.84,129.78,12.89
5996,31-08-2012,545844.91,0,71.93,3.88,138.38,9.42
5997,02-12-2011,945267.68,0,26.47,3.39,129.85,6.62
5998,13-07-2012,979848.71,0,67.48,3.56,138.26,4.16


In [16]:
!pip install nlpk

  Obtaining dependency information for nlpk from https://files.pythonhosted.org/packages/d7/c3/189e9940b2d14a4d557f74c9cf85b55e2fb3c73f5213e55eef5efeb53c3f/nlpk-0.0.6-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/47.7 kB ? eta -:--:--
   -------- ------------------------------- 10.2/47.7 kB ? eta -:--:--
   ------------------------- -------------- 30.7/47.7 kB 325.1 kB/s eta 0:00:01
   ------------------------- -------------- 30.7/47.7 kB 325.1 kB/s eta 0:00:01
   ------------------------- -------------- 30.7/47.7 kB 325.1 kB/s eta 0:00:01
   ------------------------- -------------- 30.7/47.7 kB 325.1 kB/s eta 0:00:01
   ------------------------- -------------- 30.7/47.7 kB 325.1 kB/s eta 0:00:01
   ---------------------------------------- 47.7/47.7 kB 119.7 kB/s eta 0:00:00


In [19]:
df.isna().sum()
    

Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [20]:
df.duplicated().sum()

0

In [21]:
df.describe()

,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6.000000e+03,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,1.045170e+06,0.068333,60.709908,3.359937,171.763485,8.008930
std,5.649905e+05,0.252338,18.424211,0.458530,39.360347,1.880277
min,2.099862e+05,0.000000,-2.060000,2.470000,126.060000,3.880000
25%,5.516958e+05,0.000000,47.550000,2.940000,131.740000,6.900000
50%,9.576772e+05,0.000000,62.675000,3.450000,182.810000,7.870000
75%,1.419039e+06,0.000000,74.972500,3.740000,212.762500,8.620000
max,3.818686e+06,1.000000,100.140000,4.470000,227.230000,14.310000


In [30]:
all_column_stats = df.describe()

all_column_stats = all_column_stats.drop(['count', 'min', 'max'])

fig = sp.make_subplots(rows=3, cols=3, subplot_titles=all_column_stats.columns)

for i, column in enumerate(all_column_stats.columns):
    row_num = i // 3 + 1
    col_num = i % 3 + 1
    
    column_stats = all_column_stats[column]
    
    fig.add_trace(
        go.Bar(
            x=column_stats.index,
            y=column_stats.values,
            marker_color=px.colors.qualitative.Set3,
            texttemplate='%{y:.2s}',
            textposition='outside',
        ),
        row=row_num,
        col=col_num,
    )
    
    fig.update_layout(
#         xaxis_title=f'Descriptive Statistics for {column
        yaxis_title='Values',
    )

# Update layout for the entire figure
fig.update_layout(showlegend=False, height=500, width=1000, title_text="Descriptive Statistics")
fig.show()